In [ ]:
import pandas as pd
import json

def json_to_dataframe(file_path):
    """Converts a JSON file to a Pandas DataFrame.

    Args:
        file_path: The path to the JSON file.aaaa

    Returns:
        A Pandas DataFrame.
    """

    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
        video_id = data['watch_url'].split('=')[-1]
        data['video_id'] = video_id 
        df = pd.DataFrame([data])
    return df

directory_path = 'media-info'  

all_dataframes = []

import os
for filename in os.listdir(directory_path):
    if filename.endswith('.json'):
        file_path = os.path.join(directory_path, filename)
        df = json_to_dataframe(file_path)
        df['filename'] = filename
        all_dataframes.append(df)


final_df = pd.concat(all_dataframes, ignore_index=True)

final_df

In [ ]:
final_df.to_csv('media-info.csv', index=False)

In [ ]:
def split_text(transcript):
    result = []
    for item in transcript:
        words = item['text'].split()
        duration_per_word = item['duration'] / len(words)
        start_time = item['start']
        for word in words:
            result.append({
                'text': word,
                'start': round(start_time, 2),
                'duration': round(duration_per_word, 2)
            })
            start_time += duration_per_word
    return result

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('media-info.csv')

In [ ]:
from tqdm.auto import tqdm
from youtube_transcript_api import YouTubeTranscriptApi
import json

err_list = []
for index, row in tqdm(df.iterrows()):
    print(row['filename'][:-5], row['video_id'])
    try:
        filename = row['filename'][:-5]
        transcript = YouTubeTranscriptApi.get_transcript(row['video_id'], languages=['vi'])
        with open('./transcript/' + filename + '_VI.json', 'w', encoding='utf-8') as f:
            json.dump(transcript, f, ensure_ascii=False, indent=4)

        transcript_list = YouTubeTranscriptApi.list_transcripts(row['video_id'])
        transcript = transcript_list.find_transcript(['vi'])
        translated_transcript = transcript.translate('en')
        with open('./transcript/' + filename + '_EN.json', 'w', encoding='utf-8') as f:
            json.dump(translated_transcript.fetch(), f, ensure_ascii=False, indent=4)
    except:
        print("Error:", row['filename'][:-5], row['video_id'])
        err_list.append(row['filename'][:-5])
        continue

In [1]:
from tqdm.auto import tqdm
from youtube_transcript_api import YouTubeTranscriptApi
import json

row = {}
row['filename'] = 'L03_V020.json'
row['video_id'] = '82hGUBf6ais'

filename = row['filename'][:-5]
transcript = YouTubeTranscriptApi.get_transcript(row['video_id'], languages=['vi'])
with open('./transcript/' + filename + '_VI.json', 'w', encoding='utf-8') as f:
    json.dump(transcript, f, ensure_ascii=False, indent=4)

transcript_list = YouTubeTranscriptApi.list_transcripts(row['video_id'])
transcript = transcript_list.find_transcript(['vi'])
translated_transcript = transcript.translate('en')
with open('./transcript/' + filename + '_EN.json', 'w', encoding='utf-8') as f:
    json.dump(translated_transcript.fetch(), f, ensure_ascii=False, indent=4)